# Hourly dataset for the project 


### NORD MACROZONE

In [1]:
import sys
print(sys.executable)

c:\ProgramData\anaconda3\python.exe


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_nord = qh[qh['MACROZONA'] == 'NORD']


qh_nord.tail()

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,MACROZONA,SBIL_MWH
ORAINI,,
2024-12-18 15:15:00,NORD,91.954
2024-12-18 15:30:00,NORD,25.418
2024-12-18 15:45:00,NORD,-8.331
2024-12-18 16:00:00,NORD,80.467
2024-12-18 16:15:00,NORD,146.050


In [3]:
# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_nord = qh_nord.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_nord['MACROZONA'] = h_nord['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
h_nord.tail()

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_16544\4018051853.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  h_nord = qh_nord.resample('H').sum()


,SBIL_MWH
ORAINI,
2024-12-18 12:00:00,985.019
2024-12-18 13:00:00,560.154
2024-12-18 14:00:00,371.341
2024-12-18 15:00:00,270.372
2024-12-18 16:00:00,226.517


In [4]:
lags = [-1, -2, -3, -24]

# Create a new DataFrame with SBIL_MWH and its lagged values
df_sbil_lagged = pd.DataFrame(h_nord['SBIL_MWH'])

# Add lagged columns
for lag in lags:
    df_sbil_lagged[f'SBIL_MWH_lag{abs(lag)}'] = df_sbil_lagged['SBIL_MWH'].shift(-lag)

df_sbil_lagged = df_sbil_lagged.drop('SBIL_MWH', axis=1)
df_sbil_lagged = df_sbil_lagged.resample('H').sum()

df_sbil_lagged

C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_16544\1260965639.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sbil_lagged = df_sbil_lagged.resample('H').sum()


,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24
ORAINI,,,,
2019-01-01 00:00:00,0.000,0.000,0.000,0.000
2019-01-01 01:00:00,-111.900,0.000,0.000,0.000
2019-01-01 02:00:00,-112.451,-111.900,0.000,0.000
2019-01-01 03:00:00,-88.849,-112.451,-111.900,0.000
2019-01-01 04:00:00,127.792,-88.849,-112.451,0.000
...,...,...,...,...
2024-12-18 12:00:00,833.325,728.298,762.205,51.051
2024-12-18 13:00:00,985.019,833.325,728.298,-617.973
2024-12-18 14:00:00,560.154,985.019,833.325,-667.040


In [5]:
# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME,FLOWQUARTER,TIMERESOLUTION
ORAINI,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,23521.449,24243.696,2372.707,4174.117,11460.324,901.738,1679.169,2148.394,0.0,None,...,2438.000,0.0,0.0,187.0,0.0,24056.696,23730.449,2018-12-31 13:51:37.097,NaN,None
2019-01-01 01:00:00,22417.488,23396.353,2245.300,3885.330,10964.464,861.702,1571.783,2015.909,0.0,None,...,2342.000,0.0,0.0,187.0,0.0,23209.353,22588.488,2018-12-31 13:51:37.097,NaN,None
2019-01-01 02:00:00,21496.678,22353.623,2145.655,3606.687,10604.767,829.694,1474.098,1888.777,0.0,None,...,2175.000,0.0,0.0,0.0,0.0,22353.623,22057.240,2018-12-31 13:51:37.097,NaN,None
2019-01-01 03:00:00,20322.418,20576.217,2050.474,3392.350,10234.366,805.191,1401.924,1804.113,0.0,None,...,2310.000,0.0,0.0,0.0,0.0,20576.217,20468.106,2018-12-31 13:51:37.100,NaN,None
2019-01-01 04:00:00,19979.994,21238.994,2016.645,3314.058,10088.680,791.251,1366.832,1769.976,0.0,None,...,2351.000,433.0,0.0,187.0,0.0,21051.994,19979.994,2018-12-31 13:51:37.100,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-13 19:00:00,43210.855,44069.484,3675.566,7793.277,23323.551,1174.003,2364.969,2613.840,0.0,None,...,477.000,0.0,0.0,0.0,0.0,44010.515,43510.555,2024-12-12 13:10:49.317,20.0,PT60
2024-12-13 20:00:00,40943.994,40943.994,3403.500,7425.759,21669.275,1153.128,2320.965,2515.856,0.0,None,...,784.000,0.0,0.0,0.0,625.0,40172.271,40943.994,2024-12-12 13:10:49.317,21.0,PT60
2024-12-13 21:00:00,37181.938,37181.938,3108.442,6828.312,20012.009,1075.712,2164.911,2326.693,0.0,None,...,1623.000,0.0,0.0,0.0,533.0,36149.383,37181.938,2024-12-12 13:10:49.317,22.0,PT60


In [6]:
mgp_volumes_nord = mgp_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
mgp_volumes_nord.rename(columns={"NORD_PURCHASES": "MGP_NORD_PURCHASES", "NORD_SALES": "MGP_NORD_SALES"}, inplace=True)
mgp_volumes_nord = mgp_volumes_nord[~mgp_volumes_nord.index.duplicated(keep='first')]


#mi1_volumes = volumes[volumes['MARKET'] == 'MI1']
#mi1_volumes = mi1_volumes.drop(columns=['MARKET'])
#mi1_volumes_nord = mi1_volumes[['NORD_PURCHASES', 'NORD_SALES']].copy()
#mi1_volumes_nord.rename(columns={"NORD_PURCHASES": "MI1_NORD_PURCHASES", "NORD_SALES": "MI1_NORD_SALES"}, inplace=True)
#mi1_volumes_nord = mi1_volumes_nord[~mi1_volumes_nord.index.duplicated(keep='first')]

In [7]:
#mi1_volumes_nord.tail(40)

In [8]:
# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16_
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = """
SELECT * FROM POWER_UNBALANCE
"""
power_curve = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
power_curve = power_curve.sort_values(by='TIMESTAMP')

power_curve = power_curve.pivot(index='TIMESTAMP', columns='SOURCE_ZONE', values='UNBALANCE_kW')

# Rename columns for clarity (optional, if needed)
power_curve.columns = [f"UNBALANCE_{col}" for col in power_curve.columns]

# Display the resulting DataFrame
power_curve = power_curve.resample('H').sum()

power_curve = power_curve[["UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD", "UNBALANCE_IDRO-PROGRAMMABILE_NORD", "UNBALANCE_SOLARE_NORD"]]

power_curve = power_curve.rename_axis("ORAINI")

power_curve

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_16544\3830827227.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  power_curve = power_curve.resample('H').sum()


,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,
2023-11-01 01:00:00,0.00,12447.43,0.000
2023-11-01 02:00:00,0.00,12130.31,0.000
2023-11-01 03:00:00,0.00,12297.07,0.000
2023-11-01 04:00:00,0.00,25895.64,0.000
2023-11-01 05:00:00,0.00,-28886.10,0.000
...,...,...,...
2024-12-18 12:00:00,64335.13,4290.63,189430.093
2024-12-18 13:00:00,51566.11,7085.20,199658.392
2024-12-18 14:00:00,30436.02,2899.60,51566.403


In [9]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM ENTSOE_DATA" 
entsoe_data = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
entsoe_data = entsoe_data.sort_values(by='ORAINI')

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


In [10]:
entsoe_data = entsoe_data[entsoe_data['DOMAIN'] == 'LOAD']
entsoe_data = entsoe_data.drop(columns=['DOMAIN'])
# Convert ORAINI to datetime using the correct method
entsoe_data['ORAINI'] = pd.to_datetime(entsoe_data['ORAINI'], format='%Y%m%d%H%M')
entsoe_data.set_index('ORAINI', inplace=True)

day_ahead_load = entsoe_data[entsoe_data['SCOPE'] == 'DAY AHEAD']
day_ahead_load = day_ahead_load[['CODZONA', 'VALUE']]
day_ahead_load["FORECAST_TOTAL_LOAD_MW"] = day_ahead_load["VALUE"]
day_ahead_load = day_ahead_load.drop(columns="VALUE")

actual_load = entsoe_data[entsoe_data['SCOPE'] == 'ACTUAL']
actual_load = actual_load[['CODZONA', 'VALUE']]
actual_load["TOTAL_LOAD_MW"] = actual_load["VALUE"]
actual_load = actual_load.drop(columns="VALUE")

entsoe_load = pd.merge(day_ahead_load, actual_load, on=['ORAINI', 'CODZONA'])

subset_zonas = ['NORD']
mnord_load = entsoe_load[entsoe_load['CODZONA'].isin(subset_zonas)]
mnord_load = mnord_load.drop(columns=['CODZONA'])
mnord_load


,FORECAST_TOTAL_LOAD_MW,TOTAL_LOAD_MW
ORAINI,,
2019-01-01 00:00:00,11110.0,11511.0
2019-01-01 01:00:00,10675.0,11039.0
2019-01-01 02:00:00,10080.0,10430.0
2019-01-01 03:00:00,9793.0,9811.0
2019-01-01 04:00:00,9485.0,9474.0
...,...,...
2024-12-18 10:00:00,25188.0,25239.0
2024-12-18 11:00:00,24729.0,24805.0
2024-12-18 12:00:00,22771.0,23063.0


In [11]:
from functools import reduce
# List of all the DataFrames to be merged
dataframes = [h_nord, df_sbil_lagged, power_curve] #mi1_volumes_nord, mnord_load, mgp_volumes_nord,
# Use reduce to merge all DataFrames on 'ORAINI'
df_nord_h_project = reduce(lambda left, right: pd.merge(left, right, on='ORAINI', how='outer'), dataframes)

#df_nord_h = df_nord.drop(columns="MACROZONA")
df_nord_h_project = df_nord_h_project[df_nord_h_project.index >= '2024-08-27']
df_nord_h_project

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,
2024-08-27 00:00:00,-6.856,-97.279,-51.994,-409.274,-270.368,-282467.38,0.00,0.000
2024-08-27 01:00:00,35.347,-6.856,-97.279,-51.994,-147.570,-287513.36,0.00,0.000
2024-08-27 02:00:00,-32.585,35.347,-6.856,-97.279,-214.758,-352965.19,0.00,0.000
2024-08-27 03:00:00,-117.998,-32.585,35.347,-6.856,21.760,-240815.93,0.00,0.000
2024-08-27 04:00:00,-297.574,-117.998,-32.585,35.347,-157.722,-255695.64,0.00,0.000
...,...,...,...,...,...,...,...,...
2024-12-18 12:00:00,985.019,833.325,728.298,762.205,51.051,64335.13,4290.63,189430.093
2024-12-18 13:00:00,560.154,985.019,833.325,728.298,-617.973,51566.11,7085.20,199658.392
2024-12-18 14:00:00,371.341,560.154,985.019,833.325,-667.040,30436.02,2899.60,51566.403


In [12]:
df_nord_h_project.tail(48)

,SBIL_MWH,SBIL_MWH_lag1,SBIL_MWH_lag2,SBIL_MWH_lag3,SBIL_MWH_lag24,UNBALANCE_IDRO-NON-PROGRAMMABILE_MACRONORD,UNBALANCE_IDRO-PROGRAMMABILE_NORD,UNBALANCE_SOLARE_NORD
ORAINI,,,,,,,,
2024-12-16 17:00:00,47.597,27.324,-42.039,514.488,-192.654,131264.22,-34343.88,-289.786
2024-12-16 18:00:00,304.595,47.597,27.324,-42.039,-57.385,52307.62,27546.41,4.800
2024-12-16 19:00:00,238.895,304.595,47.597,27.324,316.784,37409.70,20372.43,4.800
2024-12-16 20:00:00,-123.086,238.895,304.595,47.597,179.379,-8869.91,-44774.98,4.800
2024-12-16 21:00:00,-63.929,-123.086,238.895,304.595,108.179,-40630.65,26787.93,4.800
2024-12-16 22:00:00,360.948,-63.929,-123.086,238.895,-46.938,20412.03,14047.37,4.800
2024-12-16 23:00:00,323.697,360.948,-63.929,-123.086,-85.786,-24815.21,10815.98,4.800
2024-12-17 00:00:00,-94.883,323.697,360.948,-63.929,-495.647,-84935.68,27810.37,4.800
2024-12-17 01:00:00,-431.316,-94.883,323.697,360.948,-649.403,-119936.02,7275.59,4.800


In [13]:
from pathlib import Path 
filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
df_nord_h_project.to_csv(filepath)

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
C:\Users\samueledelia\AppData\Local\Temp\2\ipykernel_16544\2037195458.py:2: SyntaxWarning: invalid escape sequence '\i'
  filepath = Path('C:\imbalance_forecast\data\df_nord_h_project.csv')


### SUD MACROZONE

In [14]:
"""
import sys
print(sys.executable)"""

'\nimport sys\nprint(sys.executable)'

In [15]:
"""import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the source directory to the system path
sys.path.append(os.path.abspath('../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib

# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" 
qh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
qh = qh.sort_values(by='ORAINI')
# Filter the DataFrame for the relevant columns and rows
qh = qh[['MACROZONA', 'ORAINI', 'SBIL_MWH']]
# Convert ORAINI to datetime using the correct method
qh['ORAINI'] = pd.to_datetime(qh['ORAINI'], format='%Y%m%d%H%M')
qh.set_index('ORAINI', inplace=True)
# Nord
qh_sud = qh[qh['MACROZONA'] == 'SUD']


qh_sud.tail()"""

'import os\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Add the source directory to the system path\nsys.path.append(os.path.abspath(\'../src\'))\nfrom open_data import fetch_db_table_sqlserver16\nimport utils\nimport importlib\n\n# Reload the module\nimportlib.reload(utils)\n\n# SQL query to fetch data\nsql_query = "SELECT * FROM TERNA_SUNSET_SEGNO_QH" \nqh = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)\nqh = qh.sort_values(by=\'ORAINI\')\n# Filter the DataFrame for the relevant columns and rows\nqh = qh[[\'MACROZONA\', \'ORAINI\', \'SBIL_MWH\']]\n# Convert ORAINI to datetime using the correct method\nqh[\'ORAINI\'] = pd.to_datetime(qh[\'ORAINI\'], format=\'%Y%m%d%H%M\')\nqh.set_index(\'ORAINI\', inplace=True)\n# Nord\nqh_sud = qh[qh[\'MACROZONA\'] == \'SUD\']\n\n\nqh_sud.tail()'

In [16]:
"""# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour
h_sud = qh_sud.resample('H').sum()

# Clean the 'MACROZONA' column to retain only 'NORD'
h_sud['MACROZONA'] = h_sud['MACROZONA'].str[:4]

h_nord = h_nord.drop(columns = ['MACROZONA'])
# Display the first few rows of the resampled data
h_nord.tail()"""

"# Resample to hourly data and take the sum of 'SBIL_MWH' for each hour\nh_sud = qh_sud.resample('H').sum()\n\n# Clean the 'MACROZONA' column to retain only 'NORD'\nh_sud['MACROZONA'] = h_sud['MACROZONA'].str[:4]\n\nh_nord = h_nord.drop(columns = ['MACROZONA'])\n# Display the first few rows of the resampled data\nh_nord.tail()"